In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 172
USE = 'orders'

In [2]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [3]:
LOAD = 709
OFFSET = ''

In [4]:
users = cudf.read_parquet(f'../../data/train_with_features/train_v{VER}.pqt')
print( users.shape )
users.head()

(13465600, 5)


,user,item,pos,wgt,orders
0,11098755,626966,68,2.0,0
1,11098755,1693461,69,2.0,0
2,11098755,274783,70,2.0,0
3,11098755,1059788,71,2.0,0
4,11098755,1375597,72,2.0,0


# Add Weights

In [5]:
for k in [1,3,4,5,6,7]:

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v564_v{k}.csv')
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v564_wgt_v{k}.csv')
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{k}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{k}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

1 (13465600, 6)
3 (13465600, 7)
4 (13465600, 8)
5 (13465600, 9)
6 (13465600, 10)
7 (13465600, 11)


In [6]:
#import gc
#users = users.to_pandas()
#gc.collect()

In [7]:
USE2 = 'orders'
OFFSET = 10
for j,k in enumerate(['.../../data/candidate_scores/submission_v564_F',
                      '../../data/candidate_scores/submission_v612']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-6-22-otto/submission_v564_F (13465600, 12)
/raid/Kaggle/otto/data/submission_v612 (13465600, 13)


In [8]:
USE2 = 'clicks'
OFFSET = 12
for j,k in enumerate(['.../../data/candidate_scores/submission_v564_F',
                      '../../data/candidate_scores/submission_v612']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

../../../Dec-2022/dec-6-22-otto/submission_v564_F (13465600, 14)
/raid/Kaggle/otto/data/submission_v612 (13465600, 15)


In [9]:
#users = users.to_pandas()
#gc.collect()

In [10]:
if USE=='clicks': USE2 = 'orders'
else: USE2 = 'clicks'
    
OFFSET = 30
for j,k in enumerate(['../../data/candidate_scores/submission_v709']):

    dfA = pd.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA = cudf.DataFrame(dfA)
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(100),tmpA.shape[0]//100 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = pd.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB = cudf.DataFrame(dfB)
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(100),tmpB.shape[0]//100 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

/raid/Kaggle/otto/data0/submission_v709 (13465600, 16)


In [11]:
USE2 = 'orders'
OFFSET = 14

for j,k in enumerate( range(20,28) ):

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v564_v{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v564_wgt_v{k}.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

20 (13465600, 17)
21 (13465600, 18)
22 (13465600, 19)
23 (13465600, 20)
24 (13465600, 21)
25 (13465600, 22)
26 (13465600, 23)
27 (13465600, 24)


In [12]:
USE2 = 'orders'
OFFSET = 22
for j,k in enumerate(['../../data/candidate_scores/submission_v614']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

/raid/Kaggle/otto/data/submission_v614 (13465600, 25)


In [13]:
USE2 = 'clicks'
OFFSET = 23
for j,k in enumerate(['../../data/candidate_scores/submission_v614']):

    dfA = cudf.read_csv(f'{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'{k}_wgt.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

/raid/Kaggle/otto/data/submission_v614 (13465600, 26)


In [14]:
#USE2 = 'orders'
OFFSET = 24

for j,(k,USE2) in enumerate( zip( range(31,34),['orders','orders','clicks'] ) ):

    dfA = cudf.read_csv(f'../../data/candidate_scores/submission_v564_v{k}.csv')
    dfA = dfA.loc[dfA.session_type.str.contains(USE2)]
    dfA['user'] = dfA.session_type.str.partition('_')[0].astype('int32')
    dfA = dfA.drop('session_type',axis=1)

    tmpA = dfA.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
    tmpA['pos'] = np.tile( np.arange(50),tmpA.shape[0]//50 ).astype('int32')

    dfA = dfA[['user']]
    dfA = dfA.merge(tmpA, left_index=True, right_index=True)
    del tmpA

    dfB = cudf.read_csv(f'../../data/candidate_scores/submission_v564_wgt_v{k}.csv')
    dfB = dfB.loc[dfB.session_type.str.contains(USE2)]
    dfB['user'] = dfB.session_type.str.partition('_')[0].astype('int32')
    dfB = dfB.drop('session_type',axis=1)

    tmpB = dfB.labels.str.split(' ').explode().astype('float32').rename('item').to_frame()
    tmpB['pos'] = np.tile( np.arange(50),tmpB.shape[0]//50 ).astype('int32')

    dfB = dfB[['user']]
    dfB = dfB.merge(tmpB, left_index=True, right_index=True)
    dfB.columns = ['user',f'score_{j+OFFSET}','pos']
    del tmpB

    dfA = dfA.merge(dfB, on=['user','pos'], how='left').drop('pos',axis=1)
    dfA = dfA.loc[dfA[f'score_{j+OFFSET}']!=-2]
    del dfB
      
    #dfA = dfA.rename({'pos':f'pos_{k}'},axis=1)
    users = users.merge(dfA,on=['user','item'],how='left').fillna(-3)
    del dfA
    
    gc.collect()
    
    print(k,users.shape)

31 (13465600, 27)
32 (13465600, 28)
33 (13465600, 29)


In [15]:
users = users.to_pandas()
gc.collect()

0

In [16]:
users.head()

,user,item,pos,wgt,orders,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26
0,11098668,960325,56,3.000,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,3.0,-3.0,-3.0,2.335,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,1.0,-3.0
1,11098534,1235438,93,2.325,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.810,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0
2,11098534,1505252,91,2.400,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,4.325,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0
3,11098668,489201,63,2.400,0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.740,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0
4,11098668,1097201,62,2.500,0,-3.0,-3.0,0.5,-3.0,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,2.325,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,0.5,-3.0,1.0,-3.0,-3.0,-3.0,-3.0


# User and Item Features

In [17]:
item_features = pd.read_parquet('../../data/item_user_features/item12.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

(1825499, 7)


,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5
aid,,,,,,,
0,0.0,14,12,1.166667,0.0,1.660465,1.660465
1,0.052632,19,17,1.117647,0.229416,2.657424,2.657424
2,0.0,7,7,1.0,0.0,0.925820,0.925820
3,0.079235,732,392,1.867347,0.312539,82.133194,82.133194
4,0.0,45,37,1.216216,0.0,3.540245,3.540245


In [18]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')
del item_features
gc.collect()
users.head()

,user,item,pos,wgt,orders,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5
0,11098668,960325,56,3.000,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,3.0,-3.0,-3.0,2.335,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,1.0,-3.0,0.042471,259,221,1.171946,0.237337,17.957224,17.957224
1,11098534,1235438,93,2.325,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.810,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.118609,978,565,1.730973,0.365113,98.445999,98.445999
2,11098534,1505252,91,2.400,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,4.325,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.051374,1674,1290,1.297674,0.253584,46.664661,46.664661
3,11098668,489201,63,2.400,0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.740,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.02439,41,29,1.413793,0.156174,1.802115,1.802115
4,11098668,1097201,62,2.500,0,-3.0,-3.0,0.5,-3.0,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,2.325,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,0.5,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,0.020243,247,213,1.159624,0.167463,15.935734,15.935734


In [19]:
item_features = pd.read_parquet('../../data/item_user_features/item13.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

count_item_0_6 int32
count_item_1_6 int32
count_item_2_6 int32
count_item_3_6 int32
count_item_4_6 int32
count_item_5_6 int32
count_item_6_6 int32
count_user_0_6 int32
count_user_1_6 int32
count_user_2_6 int32
count_user_3_6 int32
count_user_4_6 int32
count_user_5_6 int32
count_user_6_6 int32
(874852, 23)


,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6
0,-1.000000,-1.0,0.000000,-1.000000,0.000000,0.000000,0.000000,-1.0,-1.0,2.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,0.755929,0.755929
2,0.000000,0.0,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,2.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,0.786796,0.786796
3,0.083333,0.0,-1.000000,0.236842,0.108696,0.047619,0.090909,60.0,18.0,-1.0,38.0,46.0,63.0,99.0,37.0,15.0,-1.0,16.0,25.0,39.0,54.0,32.294037,32.294037
4,-1.000000,-1.0,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.0,-1.0,1.0,2.0,-1.0,2.0,3.0,-1.0,-1.0,1.0,2.0,-1.0,2.0,1.0,1.214986,1.214986
11,-1.000000,-1.0,0.333333,-1.000000,-1.000000,-1.000000,0.000000,-1.0,-1.0,3.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.133893,1.133893


In [20]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')
del item_features
gc.collect()
users.head()

,user,item,pos,wgt,orders,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6
0,11098668,960325,56,3.000,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,3.0,-3.0,-3.0,2.335,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,1.0,-3.0,0.042471,259,221,1.171946,0.237337,17.957224,17.957224,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.00,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.377964,0.377964
1,11098534,1235438,93,2.325,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.810,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.118609,978,565,1.730973,0.365113,98.445999,98.445999,-1.000000,0.076923,0.068966,0.000000,0.035714,0.137931,0.14,-1.0,26.0,29.0,24.0,28.0,29.0,50.0,-1.0,16.0,25.0,18.0,21.0,22.0,27.0,14.581463,14.581463
2,11098534,1505252,91,2.400,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,4.325,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.051374,1674,1290,1.297674,0.253584,46.664661,46.664661,0.105263,0.038462,0.000000,0.071429,0.000000,0.045455,0.00,38.0,26.0,21.0,28.0,16.0,22.0,22.0,27.0,22.0,19.0,22.0,12.0,16.0,15.0,6.993194,6.993194
3,11098668,489201,63,2.400,0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.740,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.02439,41,29,1.413793,0.156174,1.802115,1.802115,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.00,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,0.487950,0.487950
4,11098668,1097201,62,2.500,0,-3.0,-3.0,0.5,-3.0,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,2.325,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,0.5,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,0.020243,247,213,1.159624,0.167463,15.935734,15.935734,0.600000,0.040000,0.142857,0.000000,0.000000,0.000000,0.00,5.0,25.0,7.0,3.0,8.0,7.0,2.0,3.0,18.0,6.0,3.0,8.0,5.0,2.0,7.755183,7.755183


In [21]:
item_features = pd.read_parquet('../../data/item_user_features/item4.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
item_features.head()

count_item int32
count_user int32
(1825499, 9)


,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
aid,,,,,,,,,
0,0.000000,33.0,26.0,54182.363281,425964.1875,1.269231,0.000000,18119.994141,142572.515625
1,0.031250,32.0,29.0,50521.250000,266521.2500,1.103448,0.176777,18789.619141,148777.078125
2,0.000000,12.0,12.0,42171.000000,286971.0000,1.000000,0.000000,11595.507812,129372.109375
3,0.079933,1201.0,668.0,50329.492188,321759.3125,1.797904,0.314015,16501.742188,167216.656250
4,0.035714,140.0,97.0,56197.734375,283306.3125,1.443299,0.186243,15681.410156,174638.421875


In [22]:
#users = users.to_pandas()

In [23]:
users = users.merge(item_features, left_on='item', right_index=True, how='left')

In [24]:
users.head()

,user,item,pos,wgt,orders,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
0,11098668,960325,56,3.000,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,3.0,-3.0,-3.0,2.335,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,1.0,-3.0,0.042471,259,221,1.171946,0.237337,17.957224,17.957224,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.00,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.058601,529.0,403.0,54074.269531,335976.71875,1.312655,0.272417,18137.787109,161549.906250
1,11098534,1235438,93,2.325,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.810,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.118609,978,565,1.730973,0.365113,98.445999,98.445999,-1.000000,0.076923,0.068966,0.000000,0.035714,0.137931,0.14,-1.0,26.0,29.0,24.0,28.0,29.0,50.0,-1.0,16.0,25.0,18.0,21.0,22.0,27.0,14.581463,14.581463,0.103797,2871.0,1636.0,52392.238281,327602.28125,1.754890,0.340664,20282.160156,180308.921875
2,11098534,1505252,91,2.400,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,4.325,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.051374,1674,1290,1.297674,0.253584,46.664661,46.664661,0.105263,0.038462,0.000000,0.071429,0.000000,0.045455,0.00,38.0,26.0,21.0,28.0,16.0,22.0,22.0,27.0,22.0,19.0,22.0,12.0,16.0,15.0,6.993194,6.993194,0.054026,3887.0,2853.0,51856.386719,313256.93750,1.362426,0.261953,19108.847656,165689.828125
3,11098668,489201,63,2.400,0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.740,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.02439,41,29,1.413793,0.156174,1.802115,1.802115,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.00,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,0.487950,0.487950,0.013158,76.0,59.0,49366.750000,299472.00000,1.288136,0.114708,20498.933594,168078.000000
4,11098668,1097201,62,2.500,0,-3.0,-3.0,0.5,-3.0,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,2.325,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,0.5,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,0.020243,247,213,1.159624,0.167463,15.935734,15.935734,0.600000,0.040000,0.142857,0.000000,0.000000,0.000000,0.00,5.0,25.0,7.0,3.0,8.0,7.0,2.0,3.0,18.0,6.0,3.0,8.0,5.0,2.0,7.755183,7.755183,0.038636,440.0,363.0,52924.816406,306037.53125,1.212121,0.215267,17908.527344,145407.390625


In [25]:
del item_features
gc.collect()

0

In [26]:
item_features = pd.read_parquet('../../data/item_user_features/item10.pqt')
for c in item_features.columns:
    if str(item_features[c].dtype)=='int32':
        print(c, item_features[c].dtype)
        item_features[c] = item_features[c].astype('float32')
print( item_features.shape )
users = users.merge(item_features, left_on='item', right_index=True, how='left')
users.head()

count_item3 int32
count_user3 int32
prev3 int32
next3 int32
orders3 int32
carts3 int32
(874852, 15)


,user,item,pos,wgt,orders,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
0,11098668,960325,56,3.000,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,3.0,-3.0,-3.0,2.335,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,1.0,-3.0,0.042471,259,221,1.171946,0.237337,17.957224,17.957224,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.00,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.058601,529.0,403.0,54074.269531,335976.71875,1.312655,0.272417,18137.787109,161549.906250,0.000000,1.0,1.0,60513.000000,60513.00000,1.000000,-1.000000,-1.000000,-1.000000,1.0,1.0,0.0,0.0,-1.0,-1.000000
1,11098534,1235438,93,2.325,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.810,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.118609,978,565,1.730973,0.365113,98.445999,98.445999,-1.000000,0.076923,0.068966,0.000000,0.035714,0.137931,0.14,-1.0,26.0,29.0,24.0,28.0,29.0,50.0,-1.0,16.0,25.0,18.0,21.0,22.0,27.0,14.581463,14.581463,0.103797,2871.0,1636.0,52392.238281,327602.28125,1.754890,0.340664,20282.160156,180308.921875,0.086022,186.0,126.0,52062.953125,312191.96875,1.476190,0.317283,17268.580078,173418.171875,71.0,65.0,2.0,12.0,1.0,1.000000
2,11098534,1505252,91,2.400,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,4.325,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.051374,1674,1290,1.297674,0.253584,46.664661,46.664661,0.105263,0.038462,0.000000,0.071429,0.000000,0.045455,0.00,38.0,26.0,21.0,28.0,16.0,22.0,22.0,27.0,22.0,19.0,22.0,12.0,16.0,15.0,6.993194,6.993194,0.054026,3887.0,2853.0,51856.386719,313256.93750,1.362426,0.261953,19108.847656,165689.828125,0.046243,173.0,133.0,51397.167969,315591.37500,1.300752,0.210620,18377.238281,170380.343750,82.0,79.0,0.0,8.0,-1.0,1.142857
3,11098668,489201,63,2.400,0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.740,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.02439,41,29,1.413793,0.156174,1.802115,1.802115,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.00,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,0.487950,0.487950,0.013158,76.0,59.0,49366.750000,299472.00000,1.288136,0.114708,20498.933594,168078.000000,0.000000,5.0,5.0,42443.199219,284363.18750,1.000000,0.000000,13942.373047,232101.656250,4.0,4.0,0.0,0.0,-1.0,-1.000000
4,11098668,1097201,62,2.500,0,-3.0,-3.0,0.5,-3.0,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,2.325,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,0.5,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,0.020243,247,213,1.159624,0.167463,15.935734,15.935734,0.600000,0.040000,0.142857,0.000000,0.000000,0.000000,0.00,5.0,25.0,7.0,3.0,8.0,7.0,2.0,3.0,18.0,6.0,3.0,8.0,5.0,2.0,7.755183,7.755183,0.038636,440.0,363.0,52924.816406,306037.53125,1.212121,0.215267,17908.527344,145407.390625,0.087719,57.0,45.0,49858.070312,374237.03125,1.266667,0.285401,20679.736328,172058.781250,31.0,36.0,0.0,5.0,-1.0,1.250000


In [27]:
del item_features
gc.collect()

0

In [28]:
user_features = pd.read_parquet('../../data/item_user_features/user4.pqt')
for c in user_features.columns:
    if str(user_features[c].dtype)=='int32':
        print(c, user_features[c].dtype)
        user_features[c] = user_features[c].astype('float32')
print( user_features.shape )
user_features.head()

count_item2 int32
unique_item2 int32
(1801251, 9)


,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
session,,,,,,,,,
11098528,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000
11098529,0.000000,1.0,1.0,79200.000000,338400.00000,1.000000,-1.000000,-1.000000,-1.000000
11098530,0.166667,6.0,2.0,79665.835938,338865.84375,3.000000,0.408248,547.326355,547.326355
11098531,0.333333,24.0,11.0,79429.914062,338629.90625,2.181818,0.761387,174.236954,174.236954
11098532,0.000000,2.0,2.0,79598.500000,338798.50000,1.000000,0.000000,562.149902,562.149902


In [29]:
users = users.merge(user_features, left_on='user', right_index=True, how='left')

In [30]:
del user_features
gc.collect()

0

In [31]:
users.head()

,user,item,pos,wgt,orders,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
0,11098668,960325,56,3.000,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,3.0,-3.0,-3.0,2.335,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,1.0,-3.0,0.042471,259,221,1.171946,0.237337,17.957224,17.957224,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.00,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.058601,529.0,403.0,54074.269531,335976.71875,1.312655,0.272417,18137.787109,161549.906250,0.000000,1.0,1.0,60513.000000,60513.00000,1.000000,-1.000000,-1.000000,-1.000000,1.0,1.0,0.0,0.0,-1.0,-1.000000,0.5,2.0,1.0,79248.000000,338448.00000,2.0,0.707107,18.384777,18.384777
1,11098534,1235438,93,2.325,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.810,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.118609,978,565,1.730973,0.365113,98.445999,98.445999,-1.000000,0.076923,0.068966,0.000000,0.035714,0.137931,0.14,-1.0,26.0,29.0,24.0,28.0,29.0,50.0,-1.0,16.0,25.0,18.0,21.0,22.0,27.0,14.581463,14.581463,0.103797,2871.0,1636.0,52392.238281,327602.28125,1.754890,0.340664,20282.160156,180308.921875,0.086022,186.0,126.0,52062.953125,312191.96875,1.476190,0.317283,17268.580078,173418.171875,71.0,65.0,2.0,12.0,1.0,1.000000,0.0,7.0,5.0,79738.140625,338938.15625,1.4,0.000000,533.075806,533.075806
2,11098534,1505252,91,2.400,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,4.325,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.051374,1674,1290,1.297674,0.253584,46.664661,46.664661,0.105263,0.038462,0.000000,0.071429,0.000000,0.045455,0.00,38.0,26.0,21.0,28.0,16.0,22.0,22.0,27.0,22.0,19.0,22.0,12.0,16.0,15.0,6.993194,6.993194,0.054026,3887.0,2853.0,51856.386719,313256.93750,1.362426,0.261953,19108.847656,165689.828125,0.046243,173.0,133.0,51397.167969,315591.37500,1.300752,0.210620,18377.238281,170380.343750,82.0,79.0,0.0,8.0,-1.0,1.142857,0.0,7.0,5.0,79738.140625,338938.15625,1.4,0.000000,533.075806,533.075806
3,11098668,489201,63,2.400,0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.740,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.02439,41,29,1.413793,0.156174,1.802115,1.802115,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.00,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,0.487950,0.487950,0.013158,76.0,59.0,49366.750000,299472.00000,1.288136,0.114708,20498.933594,168078.000000,0.000000,5.0,5.0,42443.199219,284363.18750,1.000000,0.000000,13942.373047,232101.656250,4.0,4.0,0.0,0.0,-1.0,-1.000000,0.5,2.0,1.0,79248.000000,338448.00000,2.0,0.707107,18.384777,18.384777
4,11098668,1097201,62,2.500,0,-3.0,-3.0,0.5,-3.0,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,2.325,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,0.5,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,0.020243,247,213,1.159624,0.167463,15.935734,15.935734,0.600000,0.040000,0.142857,0.000000,0.000000,0.000000,0.00,5.0,25.0,7.0,3.0,8.0,7.0,2.0,3.0,18.0,6.0,3.0,8.0,5.0,2.0,7.755183,7.755183,0.038636,440.0,363.0,52924.816406,306037.53125,1.212121,0.215267,17908.527344,145407.390625,0.087719,57.0,45.0,49858.070

In [32]:
for c in users.columns:
    print(c,users[c].dtype)

user int32
item int32
pos int32
wgt float32
orders int8
score_1 float32
score_3 float32
score_4 float32
score_5 float32
score_6 float32
score_7 float32
score_10 float32
score_11 float32
score_12 float32
score_13 float32
score_30 float32
score_14 float32
score_15 float32
score_16 float32
score_17 float32
score_18 float32
score_19 float32
score_20 float32
score_21 float32
score_22 float32
score_23 float32
score_24 float32
score_25 float32
score_26 float32
buy_ratio_5 Float32
count_item_5 Int32
count_user_5 Int32
repeat_5 Float32
buy_ratio_std_5 Float32
item_day_std_5 float32
user_day_std_5 float32
buy_ratio_0_6 float32
buy_ratio_1_6 float32
buy_ratio_2_6 float32
buy_ratio_3_6 float32
buy_ratio_4_6 float32
buy_ratio_5_6 float32
buy_ratio_6_6 float32
count_item_0_6 float32
count_item_1_6 float32
count_item_2_6 float32
count_item_3_6 float32
count_item_4_6 float32
count_item_5_6 float32
count_item_6_6 float32
count_user_0_6 float32
count_user_1_6 float32
count_user_2_6 float32
count_user_3_

In [33]:
for c in users.columns:
    t = str(users[c].dtype)
    if (t=='int64'): 
        users[c] = users[c].astype('int32')
        print('convert',c,'to int32')
    if (t=='float64')|(t=='Int32')|(t=='Float32'):
        users[c] = users[c].astype('float32')
        print('convert',c,'to float32')

convert buy_ratio_5 to float32
convert count_item_5 to float32
convert count_user_5 to float32
convert repeat_5 to float32
convert buy_ratio_std_5 to float32


In [34]:
#%%time
#users = users.sort_values(['user','pos']).reset_index(drop=True)

In [35]:
#users = users.reset_index(drop=True)
#users = users.to_pandas()
#gc.collect()

In [36]:
users.head()

,user,item,pos,wgt,orders,score_1,score_3,score_4,score_5,score_6,score_7,score_10,score_11,score_12,score_13,score_30,score_14,score_15,score_16,score_17,score_18,score_19,score_20,score_21,score_22,score_23,score_24,score_25,score_26,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5,buy_ratio_0_6,buy_ratio_1_6,buy_ratio_2_6,buy_ratio_3_6,buy_ratio_4_6,buy_ratio_5_6,buy_ratio_6_6,count_item_0_6,count_item_1_6,count_item_2_6,count_item_3_6,count_item_4_6,count_item_5_6,count_item_6_6,count_user_0_6,count_user_1_6,count_user_2_6,count_user_3_6,count_user_4_6,count_user_5_6,count_user_6_6,item_day_std_6,user_day_std_6,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3,buy_ratio2,count_item2,unique_item2,hour_mean2,day_mean2,repeat2,buy_ratio_std2,hour_std2,day_std2
0,11098668,960325,56,3.000,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,3.0,-3.0,-3.0,2.335,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.0,1.0,-3.0,0.042471,259.0,221.0,1.171946,0.237337,17.957224,17.957224,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.00,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,0.377964,0.377964,0.058601,529.0,403.0,54074.269531,335976.71875,1.312655,0.272417,18137.787109,161549.906250,0.000000,1.0,1.0,60513.000000,60513.00000,1.000000,-1.000000,-1.000000,-1.000000,1.0,1.0,0.0,0.0,-1.0,-1.000000,0.5,2.0,1.0,79248.000000,338448.00000,2.0,0.707107,18.384777,18.384777
1,11098534,1235438,93,2.325,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.810,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.118609,978.0,565.0,1.730973,0.365113,98.445999,98.445999,-1.000000,0.076923,0.068966,0.000000,0.035714,0.137931,0.14,-1.0,26.0,29.0,24.0,28.0,29.0,50.0,-1.0,16.0,25.0,18.0,21.0,22.0,27.0,14.581463,14.581463,0.103797,2871.0,1636.0,52392.238281,327602.28125,1.754890,0.340664,20282.160156,180308.921875,0.086022,186.0,126.0,52062.953125,312191.96875,1.476190,0.317283,17268.580078,173418.171875,71.0,65.0,2.0,12.0,1.0,1.000000,0.0,7.0,5.0,79738.140625,338938.15625,1.4,0.000000,533.075806,533.075806
2,11098534,1505252,91,2.400,0,-3.0,-3.0,-3.0,1.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,4.325,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.0,-3.0,1.0,-3.0,0.051374,1674.0,1290.0,1.297674,0.253584,46.664661,46.664661,0.105263,0.038462,0.000000,0.071429,0.000000,0.045455,0.00,38.0,26.0,21.0,28.0,16.0,22.0,22.0,27.0,22.0,19.0,22.0,12.0,16.0,15.0,6.993194,6.993194,0.054026,3887.0,2853.0,51856.386719,313256.93750,1.362426,0.261953,19108.847656,165689.828125,0.046243,173.0,133.0,51397.167969,315591.37500,1.300752,0.210620,18377.238281,170380.343750,82.0,79.0,0.0,8.0,-1.0,1.142857,0.0,7.0,5.0,79738.140625,338938.15625,1.4,0.000000,533.075806,533.075806
3,11098668,489201,63,2.400,0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,-3.00,-3.0,-3.0,-3.0,2.740,-3.0,-3.0,-3.00,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,1.0,-3.0,0.024390,41.0,29.0,1.413793,0.156174,1.802115,1.802115,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.00,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,0.487950,0.487950,0.013158,76.0,59.0,49366.750000,299472.00000,1.288136,0.114708,20498.933594,168078.000000,0.000000,5.0,5.0,42443.199219,284363.18750,1.000000,0.000000,13942.373047,232101.656250,4.0,4.0,0.0,0.0,-1.0,-1.000000,0.5,2.0,1.0,79248.000000,338448.00000,2.0,0.707107,18.384777,18.384777
4,11098668,1097201,62,2.500,0,-3.0,-3.0,0.5,-3.0,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,2.325,-3.0,-3.0,0.05,-3.0,-3.0,-3.0,0.5,-3.0,1.0,-3.0,-3.0,-3.0,-3.0,0.020243,247.0,213.0,1.159624,0.167463,15.935734,15.935734,0.600000,0.040000,0.142857,0.000000,0.000000,0.000000,0.00,5.0,25.0,7.0,3.0,8.0,7.0,2.0,3.0,18.0,6.0,3.0,8.0,5.0,2.0,7.755183,7.755183,0.038636,440.0,363.0,52924.816406,306037.53125,1.212121,0.215267,17908.527344,145407.390625,0.08771

In [37]:
users.to_parquet(f'../../data/train_with_features/train_v{VER}.pqt',index=False)